<a href="https://colab.research.google.com/github/karishita/MT2025011_MT2025052_Classification/blob/main/Generate_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import joblib
#gender
def gender_mapping(gender):
  if(gender=='Female'):
    return 1
  else:
    return 0
#CAEC
def CAEC_mapping(value):
  if(value=='Sometimes'):
    return 0
  elif(value=='Frequently'):
    return 1
  elif(value=='Always'):
    return 3
  elif(value=='no'):
    return 2
#CALC
def CALC_mapping(value):
  if(value=='no'):
    return 2
  elif(value=='Sometimes'):
    return 0
  elif(value=='Frequently'):
    return 1
  elif(value=='Always'):
    return 3
#MTRANS
def MTRANS_mapping(value):
  if(value=='Public_Transportation'):
    return 0
  elif(value=='Walking'):
    return 2
  elif(value=='Automobile'):
    return 1
  elif(value=='Motorbike'):
    return 3
  elif(value=='Bike'):
    return 4
#target
def reverse_target_mapping(value):
    if value == 1:
        return 'Normal_Weight'
    elif value == 2:
        return 'Overweight_Level_I'
    elif value == 3:
        return 'Overweight_Level_II'
    elif value == 4:
        return 'Obesity_Type_I'
    elif value == 0:
        return 'Insufficient_Weight'
    elif value == 5:
        return 'Obesity_Type_II'
    elif value == 6:
        return 'Obesity_Type_III'

#upload the file containing test data
df=pd.read_csv('/content/test.csv')

#Mapping values
df['Gender']=df['Gender'].apply(gender_mapping)
df['family_history_with_overweight']=df['family_history_with_overweight'].apply(lambda x:1 if x=='yes' else 0)
df['FAVC']=df['FAVC'].apply(lambda x:1 if x=='yes' else 0)
df['CAEC']=df['CAEC'].apply(CAEC_mapping)
df['SMOKE']=df['SMOKE'].apply(lambda x:1 if x=='yes' else 0)
df['SCC']=df['SCC'].apply(lambda x:1 if x=='yes' else 0)
df['CALC']=df['CALC'].apply(CALC_mapping)
df['MTRANS']=df['MTRANS'].apply(MTRANS_mapping)
df['FCVC']=df['FCVC'].round().astype(int)
df['NCP']=df['NCP'].round().astype(int)
df['CH2O']=df['CH2O'].round().astype(int)
df['FAF']=df['FAF'].round().astype(int)
df['TUE']=df['TUE'].round().astype(int)

#Changing columns to categorical except Age, Weight and Height
exclude_cols = ['Age', 'Height', 'Weight']
for col in df.columns:
    if col not in exclude_cols:
        df[col] = df[col].astype('category')

#Applying standard scaler to the data using scaler object saved prior
scaler=joblib.load('/content/scaler1.pkl')
df[['Age', 'Weight', 'Height']] = scaler.transform(df[['Age', 'Weight', 'Height']])

#Use the saved trained model to generate predictions
best_xgb= joblib.load('/content/xgb_model.pkl')
y_pred=best_xgb.predict(df.iloc[:,1:])
y_pred_labels = pd.Series(y_pred).apply(reverse_target_mapping)
df['WeightCategory']=y_pred_labels
df_new=df[['id','WeightCategory']]
df_new.to_csv('submission.csv',index=False)
